## FD ANN SINGLE DISTANCE (Hb2)

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/MCX_data'
sys.path.append(folder_path)

Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
import os
import numpy as np
import sys
import pickle
from tqdm import tqdm
! pip install pmcx
from concurrent.futures import ThreadPoolExecutor
from sklearn.preprocessing import StandardScaler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 87.0 MB/s eta 0:00:00


### Read the saved CSV

In [ ]:
import pandas as pd
import glob
import numpy as np
import sys
! pip install pmcx
from google.colab import drive
drive.mount('/content/drive')
folder_path = '/content/drive/MyDrive/MCX_data'
sys.path.append(folder_path)
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
csv_path = '/content/drive/MyDrive/MCX_data/stage2_train.csv'
final_array = np.loadtxt(csv_path, delimiter=',')
print(final_array.shape)
X_train = final_array
#
csv_path_test = '/content/drive/MyDrive/MCX_data/stage2_test.csv'
final_array = np.loadtxt(csv_path_test, delimiter=',')
print(final_array.shape)
X_test = final_array

(10000, 192)
(1000, 192)


### GT

In [ ]:
GT_folder_train = '/content/drive/MyDrive/MCX_data/stage2_csv/'
GT_folder_test = '/content/drive/MyDrive/MCX_data/test_csv/'

In [ ]:
csv_files_train = glob.glob(os.path.join(GT_folder_train, '*.csv'))
GT_all_train = pd.concat([pd.read_csv(f) for f in csv_files_train], ignore_index=True)
csv_files_test = glob.glob(os.path.join(GT_folder_test, '*.csv'))
GT_all_test = pd.concat([pd.read_csv(f) for f in csv_files_test], ignore_index=True)

In [ ]:
sorted_ids = [i+1 for i in range(10000)]
# Ensure ID column is integer
GT_all_train['ID'] = GT_all_train['ID'].astype(int)

# Filter and sort GT_all to match the sorted_ids order
GT_filtered = GT_all_train[GT_all_train['ID'].isin(sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[sorted_ids].reset_index()

# Extract HHB2 as Y
Y = GT_filtered['HHB2'].values
y_train = Y.reshape(-1, 1)

In [ ]:
sorted_ids = [i+1 for i in range(1000)]
# Ensure ID column is integer
GT_all_test['ID'] = GT_all_test['ID'].astype(int)

# Filter and sort GT_all to match the sorted_ids order
GT_filtered = GT_all_test[GT_all_test['ID'].isin(sorted_ids)].copy()
GT_filtered = GT_filtered.set_index('ID').loc[sorted_ids].reset_index()

# Extract HHB2 as Y
Y = GT_filtered['HHB2'].values
y_test = Y.reshape(-1, 1)

In [ ]:
# fit only on training data
scaler_X = StandardScaler().fit(X_train)
scaler_y = StandardScaler().fit(y_train)

# apply (same μ,σ) everywhere
X_train = scaler_X.transform(X_train)
X_test  = scaler_X.transform(X_test)
Y_train = scaler_y.transform(y_train)
Y_test = scaler_y.transform(y_test)

In [ ]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(10000, 192) (1000, 192) (10000, 1) (1000, 1)


### Seperate

In [ ]:
import numpy as np
from typing import Dict, List

def split_24x8_by_distance(
    X_train: np.ndarray,   # shape (N_train, 192)
    X_test:  np.ndarray    # shape (N_test , 192)
) -> Dict[int, List[np.ndarray]]:
    """
    Split a 192-feature matrix into 8 × 24-feature distance blocks.

    Returns
    -------
    dist_dict : dict
        Keys 1…8 → [ X_train_slice , X_test_slice ]
        Each slice has shape (N, 24).
    """
    if X_train.shape[1] != 192 or X_test.shape[1] != 192:
        raise ValueError("Expect 192 columns = 8 distances × 24 features.")

    dist_dict: Dict[int, List[np.ndarray]] = {}
    for d in range(1, 9):           # distances 1 … 8
        start = (d - 1) * 24
        end   = d * 24
        dist_dict[d] = [
            X_train[:, start:end],   # (N_train, 24)
            X_test[:,  start:end]    # (N_test , 24)
        ]
    return dist_dict
distance_data = split_24x8_by_distance(X_train, X_test)

In [ ]:
len(distance_data)

8

### ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.losses import Huber
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from pathlib import Path
import joblib, tensorflow as tf
WEIGHT_DIR = Path("/content/drive/MyDrive/MCX_data/weights")

model = Sequential([
    Input(shape=(24,)),
    Dense(1000, activation='relu'),
    Dropout(0.1),

    Dense(4096, activation='relu'),
    Dropout(0.1),

    Dense(4096, activation='relu'),
    Dropout(0.2),

    Dense(8192, activation='relu'),
    Dropout(0.2),

    Dense(8192*2, activation='relu'),
    Dropout(0.2),

    Dense(8192, activation='relu'),
    Dropout(0.2),

    Dense(4096, activation='relu'),
    Dropout(0.2),

    Dense(2048, activation='relu'),
    Dropout(0.2),

    Dense(1024, activation='relu'),
    Dropout(0.2),

    Dense(512, activation='relu'),
    Dropout(0.1),

    Dense(256, activation='relu'),
    Dropout(0.1),

    Dense(64, activation='relu'),
    Dropout(0.1),

    Dense(32, activation='relu'),
    Dropout(0.0),

    Dense(1)
])


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
Path = '/content/drive/MyDrive/MCX_data/'
for i in range (1, 9):
     model = Sequential([
    Input(shape=(24,)),
    Dense(1000, activation='relu'),
    Dropout(0.1),

    Dense(4096, activation='relu'),
    Dropout(0.1),

    Dense(4096, activation='relu'),
    Dropout(0.2),

    Dense(8192, activation='relu'),
    Dropout(0.2),

    Dense(8192*2, activation='relu'),
    Dropout(0.2),

    Dense(8192, activation='relu'),
    Dropout(0.2),

    Dense(4096, activation='relu'),
    Dropout(0.2),

    Dense(2048, activation='relu'),
    Dropout(0.2),

    Dense(1024, activation='relu'),
    Dropout(0.2),

    Dense(512, activation='relu'),
    Dropout(0.1),

    Dense(256, activation='relu'),
    Dropout(0.1),

    Dense(64, activation='relu'),
    Dropout(0.1),

    Dense(32, activation='relu'),
    Dropout(0.0),

    Dense(1)
])
     checkpoint_cb = ModelCheckpoint(
    "FD_hhb2_ANN.keras",        # filepath to save to
    monitor="val_loss",     # quantity to monitor
    mode="min",             # we want to minimize val_loss
    save_best_only=True,    # only save when val_loss decreases
    save_weights_only=False # set True if you only want weights
)


     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), loss="mse")

     history = model.fit( distance_data[i][0], Y_train, epochs=200, batch_size=32, validation_split=0.1, verbose=1, callbacks=[checkpoint_cb])
     model = tf.keras.models.load_model("FD_hhb2_ANN.keras")
     y_pred = model.predict(distance_data[i][1])
     y_pred = np.clip(y_pred, scaler_y.transform(np.array([[15]]))[0][0], scaler_y.transform(np.array([[35]]))[0][0])
     ### rescale
     y_pred = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()
     y_test = scaler_y.inverse_transform(Y_test.reshape(-1, 1)).flatten()
     mse = mean_squared_error(y_test, y_pred)
     print(f"MSE for distance {i}: {mse}")
     print(f"RMSE for distance {i}: {np.sqrt(mse)}")
     df = pd.DataFrame({
        "y_test" : y_test,
        "y_pred" : y_pred
    })
     df.to_csv(Path + f"distance_{i}_hhb2_FD.csv", index=False)

Epoch 1/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 68s 204ms/step - loss: 1.0014 - val_loss: 0.9915
Epoch 2/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 26s 93ms/step - loss: 1.0010 - val_loss: 0.9830
Epoch 3/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 27s 95ms/step - loss: 0.9676 - val_loss: 0.9107
Epoch 4/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 25s 87ms/step - loss: 0.9059 - val_loss: 0.8874
Epoch 5/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 26s 91ms/step - loss: 0.8710 - val_loss: 0.8089
Epoch 6/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.8161 - val_loss: 0.8203
Epoch 7/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 26s 93ms/step - loss: 0.8030 - val_loss: 0.7717
Epoch 8/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 25s 87ms/step - loss: 0.8111 - val_loss: 0.7626
Epoch 9/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.7976 - val_loss: 0.7952
Epoch 10/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 25s 88ms/step - loss: 0.7972 - val_loss: 0.7623
Epoch 11/200
282/282 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.7939 - val_loss: 0.7654
Epoch 12/200
282/282 